# Olympic Medal Prediction Models

This notebook demonstrates building and evaluating multiple classification models to predict whether an athlete wins a medal based on demographic and categorical features.  
We will compare Logistic Regression, Random Forest, and XGBoost, then save the best-performing model.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

## Load and Prepare Data

In [5]:
df = pd.read_csv('./data/olympics_data.csv')
df['Medal_Won'] = df['Medal'].notnull().astype(int)
df_model = df.dropna(subset=['Age', 'Sex', 'NOC', 'Sport'])
X = df_model[['Age', 'Sex', 'NOC', 'Sport']]
y = df_model['Medal_Won']

## Feature Engineering

We include the following features:
- `Age`: Athlete's age  
- `Sex`: Athlete's gender  
- `NOC`: Country code  
- `Sport`: Event sport  

We will encode categorical variables using `LabelEncoder`.


In [ ]:
X = df_model[['Age', 'Sex', 'NOC', 'Sport']]
y = df_model['Medal_Won']
le = LabelEncoder()
X['Sex']   = le.fit_transform(X['Sex'])
X['NOC']   = le.fit_transform(X['NOC'])
X['Sport'] = le.fit_transform(X['Sport'])

## Train-Test Split
Split the data into training and testing sets (70% train, 30% test) to evaluate model generalization.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Model Training and Evaluation

We train three models:
1. **Logistic Regression**  
2. **Random Forest**  
3. **XGBoost**  

For each model, we fit on training data and compute accuracy on the test set.


In [8]:
# Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
acc_lr = accuracy_score(y_test, y_pred_lr)
print(f'Logistic Regression Accuracy: {acc_lr:.4f}')

# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
acc_rf = accuracy_score(y_test, y_pred_rf)
print(f'Random Forest Accuracy: {acc_rf:.4f}')

# XGBoost
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)
acc_xgb = accuracy_score(y_test, y_pred_xgb)
print(f'XGBoost Accuracy: {acc_xgb:.4f}')

Logistic Regression Accuracy: 0.8523
Random Forest Accuracy: 0.8589


c:\Users\91962\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [02:49:36] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost Accuracy: 0.8670


## Compare Models and Save Best

Create a summary table of accuracies and save the model with the highest score using `joblib`.


In [9]:
results = {
    'Logistic Regression': acc_lr,
    'Random Forest':        acc_rf,
    'XGBoost':              acc_xgb
}
best_model_name = max(results, key=results.get)
print(f"Best Model: {best_model_name} with accuracy {results[best_model_name]:.4f}")

# Map names to objects and save
models = {
    'Logistic Regression': lr,
    'Random Forest':        rf,
    'XGBoost':              xgb
}
best_model = models[best_model_name]
import os
os.makedirs('../model', exist_ok=True)
joblib.dump(best_model, '../model/best_model.pkl')
print('Best model saved to ../model/best_model.pkl')


Best Model: XGBoost with accuracy 0.8670
Best model saved to ../model/best_model.pkl


## Detailed Classification Report for Best Model

Inspect precision, recall, and F1-score to understand model performance across classes.


In [10]:
y_pred_best = best_model.predict(X_test)
print(classification_report(y_test, y_pred_best))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred_best))

              precision    recall  f1-score   support

           0       0.87      0.99      0.93     66901
           1       0.70      0.17      0.27     11592

    accuracy                           0.87     78493
   macro avg       0.79      0.58      0.60     78493
weighted avg       0.85      0.87      0.83     78493

Confusion Matrix:
[[66072   829]
 [ 9614  1978]]
